# First Steps
This Tutorial will give a short introduction on how to use Loads Kernel. For more information on the theoretical background, please consult the User Guide. It is also assumed that you sucessfully installed the software as a Python package as described in the [README](https://github.com/DLR-AE/LoadsKernel#installation--use).

To Do / Notes to myself
- Jupyter Notebooks can be tested using the nbmake plugin for pytest, see https://pypi.org/project/nbmake/.
To figure out if the notebooks are still working, "pytest --nbmake \*.ipynb" will run a test of all notebook.
- The Discus2c examples are not published, so switch to public example when available.

Let's see if we can import the software, which is a good indictor for a sucessful installation. 

In [ ]:
from loadskernel import program_flow

## Initialization and Pre-Processing

First, initialize the main module with the job name as well as input and output folders. Please adjust to your folder structure.

The program flow is organized in a pre-, main- and post-processing step. These three steps
need to be performed sequentially, but can be performed individually. This is an advantage in case you want to repeat e.g. the main- and/or post-processing while the model itself remained unchanged. Also, the main-processing may be run on a different computer or a high performance cluster while the results are copied back to
the local file system for the post-processing step.

Now, run Loads Kernel and observe the console output below. The runtime should be a few seconds and the glider below the cumulus clouds indicates the program finished.

In [ ]:
k = program_flow.Kernel(job_name='jcl_Discus2c_B2000', pre=True, main=False, post=False,
                        path_input='../../loads-kernel-examples/Discus2c/JCLs',
                        path_output='../../loads-kernel-examples/Discus2c/output')
k.run()

## The Log
The log above should show only info messages, which give some information on the model or simply tell which step is currently performed. Other messages include warnings and errors, so it's always a good indea to check the log. The log messages are also written to a file in the output folder (defined perviously by "path_output").

## Model Viewer
As the preprocessing is finished, you should find a file "model_\*.hdf5" in the output folder. You can explore the file with tools such as "hdfview" or use the Model Viewer to visualize the model. Note that HDF5 files have a locking mechnism, so always remember to close the file again.

During the installation, a link should have been added to your system PATH, so that you can launch the the Model Viewer from the command line with "model-viewer". For this tutorial, we initialze the Model Viewer interactively from python. The Model Viewer is based on Mayavi, which is powerful for 3D visualizaions but needs a few seconds to be loaded.

After a few seconds, new window should open, possibly in the background. Load the model via File -> Load model and start exploring! Remember that this GUI is not as robust and tested as you might be used form your daily software and it can break easily (e.g. random clicking, clicking buttons twice, etc.), so be careful and you will get some nice visualizations.

The Model Viewer also allows for a visual inspection / quality control. For example, you can  
- check that the geometry of the FE model is correct and that the model is complete
- check that the mass distribution looks plausible
- compare the center of gravity, mode shapes and frequencies with reference results
- inspect the aerodynmic mesh for gaps, overlaps and aspect ratio
- confirm that the control surfaces work correctly
- etc.

In [ ]:
from modelviewer import view
m = view.Modelviewer()
m.run()

## Main Processing

The main-processing depends on the model data prepared in the pre-processing. In addition, a trim case definition is needed as input in the JCL. The first step of the main-processing is to establish a set of so-called trim
conditions. The aircraft states are set as free or fixed according to the selected maneuver and
in such a way that the system is neither over- nor under-determined. Next, the solution of this
system is computed, yielding the trimmed aircraft.

As the model hasn't changed, there is no need to run the pre-processing again, so it can be turned off while the main processing is switched on.

Once the computation finished, look for "The solution converged." in the log, which indicates that the algorithm was able to find a solution that satisfies all trim parameters. There will be two new files in the output folder: "response_\*.hdf5" and "monstations_\*.hdf5". In the response, all results are stored, such as the trim solution, the aerodynamic forces, the structural deformations, the nodal forces on the structure, etc. Depending on the model size and number of load cases, the file may become very large, especially for time domain simulations. Therefor, the second file only includes the section loads calculated at the monitoring stations. 

In [ ]:
k = program_flow.Kernel(job_name='jcl_Discus2c_B2000', pre=False, main=True, post=False,
                        path_input='../../loads-kernel-examples/Discus2c/JCLs',
                        path_output='../../loads-kernel-examples/Discus2c/output')
k.run()

## Loads Compare
To visualize the section loads interactively, the tool Loads Compare can be used. Similar to the Model Viewer, the Loads Compare tool can be launched from the command line with "loads-compare". You can try that now or, alternatively, execute the following two lines.

Load the data via File -> Load Monstations. The GUI allows you to show the section loads in 2D envelopes, for example the wing root torsional moments My plotted over the bending moments Mx. Each dot in the plot corresponds to one load case or one time slice. You can also rename the dataset, change the color and load multiple datasets, which is handy to compare for example maneuver loads with gust loads or to identify differences in loads bewteen two sizing loops. 

In [ ]:
from loadscompare import compare
c = compare.Compare()
c.run()

## Post Processing

The post-processing is dedicated to the evaluation, visualization and export of the results. The sizing load cases are identified by the convex hull for the monitoring stations specified in the JCL. The automated plotting and creation of PDF documents allows for a fast control of the results and a quick detection of erroneous data by the user. 

After the post-processing, you should see the following outputs:
- The section loads are plotted at the monitoring stations and saved in a PDF document (if specified in the JCL).
- The trim results are exported as a CSV file.
- Three more CSV files simply list the failed, the sucessful and the critical trim cases. 
- The nodal forces acting on the structure are exported in MSC.Nastran format using FORCE and MOMENT cards.

For further analyses or custom plots you can open the response file in Python, Matlab or your favorite tool that understands HDF5.

In [ ]:
k = program_flow.Kernel(job_name='jcl_Discus2c_B2000', pre=False, main=False, post=True,
                        path_input='../../loads-kernel-examples/Discus2c/JCLs',
                        path_output='../../loads-kernel-examples/Discus2c/output')
k.run()